#### Available scene list

In [1]:
import json

    
f = open('../scene.json', "r")
scene_json = json.load(f)

scene_list = list(scene_json.keys())
scene_dict = scene_json

#### Visualize trajectory on map

In [2]:
from PIL import Image
import matplotlib.pyplot as plt


def visualize_traj(s_idx):
    """
    Visualize trajectory on scene map

    Args:
        s_idx (int): identifier of scene
    """
    
    # dataset path
    ds_pth = "E:/Workspace/Datasets/iGibson-dataset/iGibson-map/"
    
    scene_id = scene_list[s_idx]

    # Read map image
    map_pth = f"{ds_pth}/{scene_id}/{scene_id}.png"
    map_img = Image.open(map_pth).convert("RGB")
    plt.imshow(map_img)

    # Read trajectory txt file
    traj_pth = f"{ds_pth}/{scene_id}/tour_cam_trajectory.txt"
    traj_fp = open(traj_pth, encoding="utf-8")
    traj_data = traj_fp.readlines()
    
    print(f"Trajectory consists {len(traj_data)} points")

    # Translate trajectory to map coordinate
    scene_trans = scene_dict[scene_id]
    
    traj_x = []
    traj_y = []

    for traj_p in traj_data:
        
        traj_p = traj_p.replace("\n", "")
        traj_p = traj_p.split(",")
        traj_x.append(float(traj_p[0]) * scene_trans[2] + scene_trans[0])
        traj_y.append(float(traj_p[1]) * scene_trans[2] + scene_trans[1])
    
    # Visualize trajectory on map    
    plt.plot(traj_x, traj_y, color="blue")
    
    # Save map image
    plt.savefig(f"{ds_pth}/{scene_id}/traj.png")
    

In [3]:
    
s_idx_list = [1, 3, 4, 5, 6]

for s_idx in s_idx_list:
    
    print(scene_list[s_idx])
    visualize_traj(s_idx)
    plt.cla() # clear the matplotlib drawing
    
plt.close("all") # close all matplotlib drawing

Beechwood_1_int
Trajectory consists 4698 points
Ihlen_0_int
Trajectory consists 3187 points
Ihlen_1_int
Trajectory consists 5009 points
Merom_0_int
Trajectory consists 2762 points
Merom_1_int
Trajectory consists 3222 points


#### Read Object Map

In [6]:
import json


def conv_obj_js2txt(js_pth, txt_pth) -> None:
    """
    Convert object map from json to txt format

    Args:
        json_pth (str): object map json file path
        txt_pth (str): object map txt file path

    """
    obj_list = []
    
    with open(js_pth, encoding="utf-8") as f:
        
        js_data = json.load(f)
        print(f"JSON file consists {len(js_data)} objects")
        
        for item in js_data:
            
            # object semantic label
            k1 = "category"
            v1 = item[k1]
            label = v1
    
            # object X, Y axis center
            k2 = "center"
            v2 = (item[k2][0], item[k2][1])
            x_center = round(v2[0], 4)
            y_center = round(v2[1], 4)
            
            # object Z axis center and scale
            k3 = "z"
            v3 = (item[k3][0], item[k3][1])
            z_center = round((v3[0] + v3[1]) / 2, 4)
            z_scale = round(abs(v3[0] - v3[1]), 4)
            
            # object X, Y axis scale
            k4 = "raw_pts"
            v4 = item[k4]
            
            x_list = [p[0] for p in v4]
            y_list = [p[1] for p in v4]

            x_max = max(x_list)
            x_min = min(x_list)
            y_max = max(y_list)
            y_min = min(y_list)
            x_scale = round(x_max - x_min, 4)
            y_scale = round(y_max - y_min, 4)
            
            obj_list.append([label, x_center, y_center, z_center, x_scale, y_scale, z_scale])

    with open(txt_pth, "w", encoding="utf-8") as f:
        
        for obj_item in obj_list:
            f.write(f"{obj_item[0]},{obj_item[1]},{obj_item[2]},{obj_item[3]},{obj_item[4]},{obj_item[5]},{obj_item[6]}\n")
    
            
def conv_scene_obj(s_idx) -> None:
    """
    Convert scene object map

    Args:
        s_idx (int): identifier of scene
    """
    
    # dataset path
    ds_pth = "E:/Workspace/Datasets/iGibson-dataset/iGibson-map/"
    
    scene_id = scene_list[s_idx]
    
    obj_js_pth = f"{ds_pth}/{scene_id}/{scene_id}.json"
    obj_txt_pth = f"{ds_pth}/{scene_id}/{scene_id}.txt"
    
    conv_obj_js2txt(obj_js_pth, obj_txt_pth)
    
    
    

In [7]:
s_idx_list = [1, 3, 4, 5, 6]

for s_idx in s_idx_list:
    
    print(scene_list[s_idx])
    conv_scene_obj(s_idx)

Beechwood_1_int
JSON file consists 97 objects
Ihlen_0_int
JSON file consists 49 objects
Ihlen_1_int
JSON file consists 109 objects
Merom_0_int
JSON file consists 68 objects
Merom_1_int
JSON file consists 95 objects


#### Visualize object on map

In [10]:
import json

from PIL import Image
import matplotlib.pyplot as plt


def read_obj_json(json_pth):
    """
    Read object information from json file

    Args:
        json_pth (str): object json file path

    Returns:
        list(list(str, float, float)): semantic label and position of objects
    """
    
    obj_list = []
    
    with open(json_pth, encoding="utf-8") as f:
        
        js_data = json.load(f)
        print(f"JSON file consists {len(js_data)} objects")
        
        for item in js_data:
            
            # object semantic label
            k1 = "category"
            v1 = item[k1]
            label = v1
    
            # object X, Y axis center
            k2 = "center"
            v2 = (item[k2][0], item[k2][1])
            x_center = round(v2[0], 4)
            y_center = round(v2[1], 4)
            
            # object Z axis center and scale
            k3 = "z"
            v3 = (item[k3][0], item[k3][1])
            z_center = round((v3[0] + v3[1]) / 2, 4)
            z_scale = round(abs(v3[0] - v3[1]), 4)
            
            # object X, Y axis scale
            k4 = "raw_pts"
            v4 = item[k4]
            
            x_list = [p[0] for p in v4]
            y_list = [p[1] for p in v4]

            x_max = max(x_list)
            x_min = min(x_list)
            y_max = max(y_list)
            y_min = min(y_list)
            x_scale = round(x_max - x_min, 4)
            y_scale = round(y_max - y_min, 4)
            
            obj_list.append([label, x_center, y_center, x_scale, y_scale, z_center, z_scale])


    return obj_list    


def visualize_obj(s_idx):
    """
    Visualize objects on scene map

    Args:
        s_idx (int): identifier of scene
    """
    
    # dataset path
    ds_pth = "E:/Workspace/Datasets/iGibson-dataset/iGibson-map/"
    
    scene_id = scene_list[s_idx]

    # Read object information from json file
    obj_pth = f"{ds_pth}/{scene_id}/{scene_id}.json"
    obj_data = read_obj_json(obj_pth)

    # Read map image
    map_pth = f"{ds_pth}/{scene_id}/{scene_id}.png"
    map_img = Image.open(map_pth).convert("RGB")
    plt.imshow(map_img)

    # Translate object position to map coordinate
    scene_trans = scene_dict[scene_id]
    
    obj_x = []
    obj_y = []

    for obj_item in obj_data:
                
        obj_label = obj_item[0]
        
        obj_x.append(float(obj_item[1]) * scene_trans[2] + scene_trans[0])
        obj_y.append(float(obj_item[2]) * scene_trans[2] + scene_trans[1])
        
    # Draw objects on scene map
    plt.scatter(obj_x, obj_y, color="red")
    
    # Save map image
    plt.savefig(f"{ds_pth}/{scene_id}/obj.png")
    
    # Output result to txt file
    output_fp = open(f"{ds_pth}/{scene_id}/obj.txt", "w", encoding="utf-8")
    
    for obj_item in obj_data:
        
        output_fp.write(f"{obj_item[0]},{obj_item[1]},{obj_item[2]},{obj_item[3]},{obj_item[4]},{obj_item[5]},{obj_item[6]}\n")
    

In [11]:
s_idx_list = [1, 3, 4, 5, 6]

for s_idx in s_idx_list:
    
    print(scene_list[s_idx])
    visualize_obj(s_idx)
    plt.cla()
    
plt.close("all")

Beechwood_1_int
JSON file consists 97 objects
Ihlen_0_int
JSON file consists 49 objects
Ihlen_1_int
JSON file consists 109 objects
Merom_0_int
JSON file consists 68 objects
Merom_1_int
JSON file consists 95 objects


#### Visualize windows on map (any specific type)

In [16]:
import json

from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle


def read_json_slabel(json_pth, s_label):
    """
    Read object with specific type from json file

    Args:
        json_pth (str): object json file path
        s_label(str): specific semantic type label, eg. window

    Returns:
        list(list(str, float, float)): semantic label and position of objects
    """
    obj_list = []
    
    with open(json_pth, encoding="utf-8") as f:
        
        js_data = json.load(f)
        
        for item in js_data:
            
            # object semantic label
            k1 = "category"
            v1 = item[k1]
            label = v1
            
            if v1 != s_label:
                continue
            
            # object X, Y axis center
            k2 = "center"
            v2 = (item[k2][0], item[k2][1])
            x_center = round(v2[0], 4)
            y_center = round(v2[1], 4)
            
            # object Z axis center and scale
            k3 = "z"
            v3 = (item[k3][0], item[k3][1])
            z_center = round((v3[0] + v3[1]) / 2, 4)
            z_scale = round(abs(v3[0] - v3[1]), 4)
            
            # object X, Y axis scale
            k4 = "raw_pts"
            v4 = item[k4]
            
            x_list = [p[0] for p in v4]
            y_list = [p[1] for p in v4]

            x_max = max(x_list)
            x_min = min(x_list)
            y_max = max(y_list)
            y_min = min(y_list)

            x_scale = round(x_max - x_min, 4)
            y_scale = round(y_max - y_min, 4)
            
            obj_list.append([label, x_center, y_center, x_scale, y_scale, z_center, z_scale])
        
    print(f"JSON file consists {len(obj_list)} {s_label}")
        

    return obj_list    


def visualize_obj_slabel(s_idx, s_label):
    """
    Visualize objects with specific type on scene map

    Args:
        s_idx (int): identifier of scene
        s_label(str): specific semantic type label, eg. window
    """
    
    # dataset path
    ds_pth = "E:/Workspace/Datasets/iGibson-dataset/iGibson-map/"
    
    fig, ax = plt.subplots()

    scene_id = scene_list[s_idx]

    # Read object json
    obj_pth = f"{ds_pth}/{scene_id}/{scene_id}.json"
    obj_data = read_json_slabel(obj_pth, s_label)

    # Read map image
    map_pth = f"{ds_pth}/{scene_id}/{scene_id}.png"
    map_img = Image.open(map_pth).convert("RGB")
    plt.imshow(map_img)

    # Translate object to map coordinate
    scene_trans = scene_dict[scene_id]
   
    for obj_item in obj_data:
                
        obj_label = obj_item[0]
        
        x_center = float(obj_item[1]) * scene_trans[2] + scene_trans[0]
        y_center = float(obj_item[2]) * scene_trans[2] + scene_trans[1]
        
        x_scale = float(obj_item[3]) * scene_trans[2]
        y_scale = float(obj_item[4]) * scene_trans[2]
        
        rectangle = Rectangle((
                                x_center - (x_scale / 2), 
                                y_center - (y_scale / 2)
                                ), 
                              x_scale, 
                              y_scale, 
                              edgecolor='none', 
                              facecolor='gold')
        
        ax.add_patch(rectangle)
        
    plt.savefig(f"{ds_pth}/{scene_id}/{s_label}.png")
    
    # Output result to txt file
    output_fp = open(f"{ds_pth}/{scene_id}/{s_label}.txt", "w", encoding="utf-8")
    
    for obj_item in obj_data:
        
        output_fp.write(f"{obj_item[0]},{obj_item[1]},{obj_item[2]},{obj_item[3]},{obj_item[4]},{obj_item[5]},{obj_item[6]}\n")
        

In [17]:
s_idx_list = [1, 3, 4, 5, 6]
s_label = "window"

for s_idx in s_idx_list:
    
    print(scene_list[s_idx])
    visualize_obj_slabel(s_idx, s_label)
    plt.cla()
    
plt.close("all")

Beechwood_1_int
JSON file consists 8 window
Ihlen_0_int
JSON file consists 3 window
Ihlen_1_int
JSON file consists 12 window
Merom_0_int
JSON file consists 6 window
Merom_1_int
JSON file consists 10 window


#### Visualize wall on map

In [10]:
from PIL import Image
import matplotlib.pyplot as plt
import xml.dom.minidom


def read_wall_xml(xml_pth):

    dom = xml.dom.minidom.parse(xml_pth)
    root = dom.documentElement

    xml_objects = root.getElementsByTagName('object')
    # print(f"{len(xml_objects)} objects")

    objects_p = []
    for xo in xml_objects:
        
        bbox = xo.getElementsByTagName('bndbox')[0]

        xmin = bbox.getElementsByTagName('xmin')[0].firstChild.data
        ymin = bbox.getElementsByTagName('ymin')[0].firstChild.data
        xmax = bbox.getElementsByTagName('xmax')[0].firstChild.data
        ymax = bbox.getElementsByTagName('ymax')[0].firstChild.data
        
        objects_p.append((int(xmin), int(ymin), int(xmax), int(ymax)))
        
    return objects_p


import os


def visualize_wall(s_idx):
    
    scene_id = scene_list[s_idx]
    
    fig = plt.figure(figsize=(20, 20))
    axes = fig.add_subplot(1, 1, 1)
    
    # Visualize map image
    ds_pth = "E:/Workspace/Datasets/iGibson-dataset/iGibson-map/"
    img_pth = f"{ds_pth}/{scene_id}/{scene_id}.png"
    img = Image.open(img_pth).convert("RGB")
    plt.imshow(img)
    
    # For each scene, visualize all area
    ds_pth = "E:/Workspace/Datasets/iGibson-dataset/iGibson-area/"
    area_pth = f"{ds_pth}/{scene_id}/"
    area_list = os.listdir(area_pth)
    
    # Output result to txt file
    ds_pth = "E:/Workspace/Datasets/iGibson-dataset/iGibson-map/"
    wall_map_pth = f"{ds_pth}/{scene_id}/wall.txt"
    wall_map_fp = open(wall_map_pth, "w", encoding="utf-8")
    
    for area in area_list:
        
        # Read wall mask from xml file
        wall_pth = f"{area_pth}/{area}/mask.xml"
        wall_data = read_wall_xml(wall_pth)
        
        # Visualize wall mask
        for wall_mask in wall_data:
            x_min = wall_mask[0]
            y_min = wall_mask[1]
            x_max = wall_mask[2]
            y_max = wall_mask[3]

            square_bound = (x_min, y_min, x_max, y_max)
            square = plt.Rectangle(xy = (square_bound[0], square_bound[1]),
                                width = square_bound[2]-square_bound[0],
                                height= square_bound[3]-square_bound[1],
                                edgecolor="green",
                                facecolor="green")
            axes.add_patch(square)
            
        # Output result to txt file
        scene_trans = scene_dict[scene_id]
        
        for wall_mask in wall_data:
        
            x_min = wall_mask[0]
            y_min = wall_mask[1]
            x_max = wall_mask[2]
            y_max = wall_mask[3]
            
            x = (x_min + x_max) / 2
            x = (x - scene_trans[0]) / scene_trans[2]
            x = round(x, 4)
            y = (y_min + y_max) / 2
            y = (y - scene_trans[1]) / scene_trans[2]
            y = round(y, 4)
            
            length = round(abs(x_max - x_min) / scene_trans[2], 4)
            width = round(abs(y_max - y_min) / scene_trans[2], 4)
            
            wall_map_fp.write(f"{x},{y},{length},{width}\n")
            
    wall_map_fp.close()
            
    ds_pth = "E:/Workspace/Datasets/iGibson-dataset/iGibson-map/"
    plt.savefig(f"{ds_pth}/{scene_id}/wall.jpg")
    

In [11]:
s_idx_list = [1, 3, 4, 5, 6]

for s_idx in s_idx_list:
    
    print(scene_list[s_idx])
    visualize_wall(s_idx)
    plt.cla() # clear the matplotlib drawing
    
plt.close("all") # close all matplotlib drawing

Beechwood_1_int
Ihlen_0_int
Ihlen_1_int
Merom_0_int
Merom_1_int


#### Visualize area / sub-scene on map

In [21]:
from PIL import Image
import matplotlib.pyplot as plt
import xml.dom.minidom


def read_area_xml(xml_pth):
    """
    Read area information from xml file

    Args:
        xml_pth (str): object xml file path

    Returns:
        list(list(int, int, int, int)): boundary of area
    """
    dom = xml.dom.minidom.parse(xml_pth)
    root = dom.documentElement

    xml_objects = root.getElementsByTagName('object')

    objects_p = []
    for xo in xml_objects:
        
        bbox = xo.getElementsByTagName('bndbox')[0]

        xmin = bbox.getElementsByTagName('xmin')[0].firstChild.data
        ymin = bbox.getElementsByTagName('ymin')[0].firstChild.data
        xmax = bbox.getElementsByTagName('xmax')[0].firstChild.data
        ymax = bbox.getElementsByTagName('ymax')[0].firstChild.data
        
        objects_p.append([int(xmin), int(ymin), int(xmax), int(ymax)])
        
    return objects_p


def visualize_area(s_idx, r_idx):
    """
    Visualize area / sub-scene on scene map

    Args:
        s_idx (int): identifier of scene
        r_idx(int): identifier of area
    """
    
    scene_id = scene_list[s_idx]

    fig = plt.figure(figsize=(20, 20))
    axes = fig.add_subplot(1, 1, 1)

    # Read map image
    ds_pth = "E:/Workspace/Datasets/iGibson-dataset/iGibson-map/"
    img_pth = f"{ds_pth}/{scene_id}/{scene_id}.png"
    img = Image.open(img_pth).convert("RGB")
    plt.imshow(img)

    # Read area boundary from xml file
    ds_pth = f"E:/Workspace/Datasets/iGibson-dataset/iGibson-area/"
    xml_pth = f"{ds_pth}/{scene_id}/{r_idx}/{scene_id}.xml"
    xml_data = read_area_xml(xml_pth)[0]

    # Draw area on scene map
    square_bound = (xml_data[0], xml_data[1], xml_data[2], xml_data[3])
    square = plt.Rectangle(xy = (square_bound[0], square_bound[1]),
                        width = square_bound[2]-square_bound[0],
                        height= square_bound[3]-square_bound[1],
                        edgecolor='blue', 
                        facecolor='none')
    axes.add_patch(square)

    # Save map image
    plt.savefig(f"{ds_pth}/{scene_id}/{r_idx}/area.png")
    

In [22]:
s_idx = 1
for r_idx in range(5):
    
    print(f"{scene_list[s_idx]} {r_idx}")
    visualize_area(s_idx, r_idx)
    plt.cla()

s_idx = 3
for r_idx in range(2):
    
    print(f"{scene_list[s_idx]} {r_idx}")    
    visualize_area(s_idx, r_idx)
    plt.cla()
    
s_idx = 4
for r_idx in range(6):
    
    print(f"{scene_list[s_idx]} {r_idx}")
    visualize_area(s_idx, r_idx)
    plt.cla()
    
s_idx = 5
for r_idx in range(3):
    
    print(f"{scene_list[s_idx]} {r_idx}")
    visualize_area(s_idx, r_idx)
    plt.cla()
    
s_idx = 6
for r_idx in range(6):
    
    print(f"{scene_list[s_idx]} {r_idx}")
    visualize_area(s_idx, r_idx)
    plt.cla()
    
plt.close("all")

Beechwood_1_int 0
Beechwood_1_int 1
Beechwood_1_int 2
Beechwood_1_int 3
Beechwood_1_int 4
Ihlen_0_int 0
Ihlen_0_int 1
Ihlen_1_int 0
Ihlen_1_int 1
Ihlen_1_int 2
Ihlen_1_int 3
Ihlen_1_int 4
Ihlen_1_int 5
Merom_0_int 0
Merom_0_int 1
Merom_0_int 2
Merom_1_int 0
Merom_1_int 1
Merom_1_int 2
Merom_1_int 3
Merom_1_int 4


C:\Users\12871\AppData\Local\Temp\ipykernel_872\2459533485.py:47: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig = plt.figure(figsize=(20, 20))


Merom_1_int 5


#### Visualize trajectory within area on map

In [23]:
import math

from PIL import Image
import matplotlib.pyplot as plt
import xml.dom.minidom


def read_area_xml(xml_pth):
    """
    Read area information from xml file

    Args:
        xml_pth (str): object xml file path

    Returns:
        list(list(int, int, int, int)): boundary of area
    """
    dom = xml.dom.minidom.parse(xml_pth)
    root = dom.documentElement

    xml_objects = root.getElementsByTagName('object')

    objects_p = []
    for xo in xml_objects:
        
        bbox = xo.getElementsByTagName('bndbox')[0]

        xmin = bbox.getElementsByTagName('xmin')[0].firstChild.data
        ymin = bbox.getElementsByTagName('ymin')[0].firstChild.data
        xmax = bbox.getElementsByTagName('xmax')[0].firstChild.data
        ymax = bbox.getElementsByTagName('ymax')[0].firstChild.data
        
        objects_p.append((int(xmin), int(ymin), int(xmax), int(ymax)))
        
    return objects_p


def visualize_area_traj(s_idx, r_idx):
    """
    Visualize trajectory within area / sub-scene on scene map

    Args:
        s_idx (int): identifier of scene
        r_idx(int): identifier of area
    """
    
    scene_id = scene_list[s_idx]

    fig = plt.figure(figsize=(20, 20))
    axes = fig.add_subplot(1, 1, 1)
    
    # Read map image
    ds_pth = "E:/Workspace/Datasets/iGibson-dataset/iGibson-map/"
    img_pth = f"{ds_pth}/{scene_id}/{scene_id}.png"
    img = Image.open(img_pth).convert("RGB")
    plt.imshow(img)

    # Read area boundary from xml file
    ds_pth = "E:/Workspace/Datasets/iGibson-dataset/iGibson-area/"
    xml_pth = f"{ds_pth}/{scene_id}/{r_idx}/{scene_id}.xml"
    xml_data = read_area_xml(xml_pth)[0]
    x_min = xml_data[0]
    y_min = xml_data[1]
    x_max = xml_data[2]
    y_max = xml_data[3]

    # Draw area on scene map
    square_bound = (x_min, y_min, x_max, y_max)
    square = plt.Rectangle(xy = (square_bound[0], square_bound[1]),
                        width = square_bound[2]-square_bound[0],
                        height= square_bound[3]-square_bound[1],
                        edgecolor="red",
                        facecolor="none")
    axes.add_patch(square)

    # Read trajectory from txt file
    ds_pth = "E:/Workspace/Datasets/iGibson-dataset/iGibson-map/"
    traj_pth = f"{ds_pth}/{scene_id}/tour_cam_trajectory.txt"
    traj_fp = open(traj_pth, encoding="utf-8")
    traj_data = traj_fp.readlines()

    # Translate trajectory to map coordinate
    traj_x = [] # For visualization
    traj_y = []
    traj_sub = [] # For output
    
    scene_trans = scene_dict[scene_id]

    for traj_pos in traj_data:
        
        traj_pos = traj_pos.replace("\n", "")
        traj_pos = traj_pos.split(",")
        
        pos_x = float(traj_pos[0]) * scene_trans[2] + scene_trans[0]
        pos_y = float(traj_pos[1]) * scene_trans[2] + scene_trans[1]
        
        normal_x = float(traj_pos[2])
        normal_y = float(traj_pos[3])
        theta = math.atan(normal_y / normal_x)
        
        if x_min < pos_x < x_max and y_min < pos_y < y_max:
            
            traj_x.append(pos_x)
            traj_y.append(pos_y)
            
            traj_sub.append((traj_pos[0], traj_pos[1], theta))
    
    plt.plot(traj_x, traj_y)

    ds_pth = "E:/Workspace/Datasets/iGibson-dataset/iGibson-area/"
    plt.savefig(f"{ds_pth}/{scene_id}/{r_idx}/area_traj.png")
    
    # Output result to txt file
    traj_sub_pth = f"{ds_pth}/{scene_id}/{r_idx}/traj_sub.txt"
    traj_sub_fp = open(traj_sub_pth, "w", encoding="utf-8")
    
    for traj_p in traj_sub:
        
        traj_sub_fp.write(f"{traj_p[0]},{traj_p[1]},{traj_p[2]}\n")


In [25]:
s_idx = 1
for r_idx in range(5):
    
    print(f"{scene_list[s_idx]} {r_idx}")    
    visualize_area_traj(s_idx, r_idx)
    plt.cla()

s_idx = 3
for r_idx in range(2):
    
    print(f"{scene_list[s_idx]} {r_idx}")    
    visualize_area_traj(s_idx, r_idx)
    plt.cla()
    
s_idx = 4
for r_idx in range(6):
    
    print(f"{scene_list[s_idx]} {r_idx}")    
    visualize_area_traj(s_idx, r_idx)
    plt.cla()
    
s_idx = 5
for r_idx in range(3):
    
    print(f"{scene_list[s_idx]} {r_idx}")    
    visualize_area_traj(s_idx, r_idx)
    plt.cla()
    
s_idx = 6
for r_idx in range(6):
    
    print(f"{scene_list[s_idx]} {r_idx}")    
    visualize_area_traj(s_idx, r_idx)
    plt.cla()
    
plt.close("all")

Beechwood_1_int 0
Beechwood_1_int 1
Beechwood_1_int 2
Beechwood_1_int 3
Beechwood_1_int 4
Ihlen_0_int 0
Ihlen_0_int 1
Ihlen_1_int 0
Ihlen_1_int 1
Ihlen_1_int 2
Ihlen_1_int 3
Ihlen_1_int 4
Ihlen_1_int 5
Merom_0_int 0
Merom_0_int 1
Merom_0_int 2
Merom_1_int 0
Merom_1_int 1
Merom_1_int 2
Merom_1_int 3
Merom_1_int 4


C:\Users\12871\AppData\Local\Temp\ipykernel_872\1967586712.py:49: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig = plt.figure(figsize=(20, 20))


Merom_1_int 5


#### Visualize object within area on map

In [49]:
import json

from PIL import Image
import matplotlib.pyplot as plt
import xml.dom.minidom


def read_area_xml(xml_pth):
    """
    Read area information from xml file

    Args:
        xml_pth (str): object xml file path

    Returns:
        list(list(int, int, int, int)): boundary of area
    """
    dom = xml.dom.minidom.parse(xml_pth)
    root = dom.documentElement

    xml_objects = root.getElementsByTagName('object')

    objects_p = []
    for xo in xml_objects:
        
        bbox = xo.getElementsByTagName('bndbox')[0]

        xmin = bbox.getElementsByTagName('xmin')[0].firstChild.data
        ymin = bbox.getElementsByTagName('ymin')[0].firstChild.data
        xmax = bbox.getElementsByTagName('xmax')[0].firstChild.data
        ymax = bbox.getElementsByTagName('ymax')[0].firstChild.data
        
        objects_p.append([int(xmin), int(ymin), int(xmax), int(ymax)])
        
    return objects_p


def read_obj_json(json_pth):
    """
    Read object information from json file

    Args:
        json_pth (str): object json file path

    Returns:
        list(list(str, float, float)): semantic label and position of objects
    """
    
    obj_list = []
    
    with open(json_pth, encoding="utf-8") as f:
        
        js_data = json.load(f)
        print(f"JSON file consists {len(js_data)} objects")
        
        for item in js_data:
            
            # object semantic label
            k1 = "category"
            v1 = item[k1]
            label = v1
    
            # object X, Y axis center
            k2 = "center"
            v2 = (item[k2][0], item[k2][1])
            x_center = round(v2[0], 4)
            y_center = round(v2[1], 4)
            
            # object Z axis center and scale
            k3 = "z"
            v3 = (item[k3][0], item[k3][1])
            z_center = round((v3[0] + v3[1]) / 2, 4)
            z_scale = round(abs(v3[0] - v3[1]), 4)
            
            # object X, Y axis scale
            k4 = "raw_pts"
            v4 = item[k4]
            
            x_list = [p[0] for p in v4]
            y_list = [p[1] for p in v4]

            x_max = max(x_list)
            x_min = min(x_list)
            y_max = max(y_list)
            y_min = min(y_list)
            x_scale = round(x_max - x_min, 4)
            y_scale = round(y_max - y_min, 4)
            
            obj_list.append([label, x_center, y_center, x_scale, y_scale, z_center, z_scale])


    return obj_list    


def visualize_area_obj(s_idx, r_idx):
    """
    Visualize objects on scene map

    Args:
        s_idx (int): identifier of scene
    """
        
    scene_id = scene_list[s_idx]
    
    fig = plt.figure(figsize=(20, 20))
    axes = fig.add_subplot(1, 1, 1)

    # Read area boundary from xml file
    ds_pth = f"E:/Workspace/Datasets/iGibson-dataset/iGibson-area2/"
    xml_pth = f"{ds_pth}/{scene_id}/{r_idx}/{scene_id}.xml"
    xml_data = read_area_xml(xml_pth)[0]
    x_min = xml_data[0]
    y_min = xml_data[1]
    x_max = xml_data[2]
    y_max = xml_data[3]

    # Read object information from json file
    ds_pth = "E:/Workspace/Datasets/iGibson-dataset/iGibson-map/"
    obj_pth = f"{ds_pth}/{scene_id}/{scene_id}.json"
    obj_data = read_obj_json(obj_pth)

    # Read map image
    map_pth = f"{ds_pth}/{scene_id}/{scene_id}.png"
    map_img = Image.open(map_pth).convert("RGB")
    plt.imshow(map_img)

    # Translate object position to map coordinate
    scene_trans = scene_dict[scene_id]
    
    obj_x = []
    obj_y = []
    
    obj_sub = []

    for obj_item in obj_data:
                
        obj_label = obj_item[0]
        
        obj_center_x = float(obj_item[1]) * scene_trans[2] + scene_trans[0]
        obj_center_y = float(obj_item[2]) * scene_trans[2] + scene_trans[1]
        
        obj_scale_x = float(obj_item[3]) * scene_trans[2]
        obj_scale_y = float(obj_item[4]) * scene_trans[2]
        
        if x_min < obj_center_x < x_max and y_min < obj_center_y < y_max:
            
            rectangle = Rectangle((
                                    obj_center_x - (obj_scale_x / 2), 
                                    obj_center_y - (obj_scale_y / 2)
                                    ), 
                                obj_scale_x, 
                                obj_scale_y, 
                                edgecolor='none', 
                                facecolor='gold')
            
            axes.add_patch(rectangle)
            
            obj_sub.append([obj_item[0],obj_item[1],obj_item[2],obj_item[3],obj_item[4],obj_item[5],obj_item[6]])
        
    print(f"Area has {len(obj_sub)} objects")

    # Save map image
    ds_pth = f"E:/Workspace/Datasets/iGibson-dataset/iGibson-area/"
    plt.savefig(f"{ds_pth}/{scene_id}/{r_idx}/area_obj.png")
    
    # Output result to txt file
    output_fp = open(f"{ds_pth}/{scene_id}/{r_idx}/map_obj.txt", "w", encoding="utf-8")
    
    for obj_item in obj_data:
        
        output_fp.write(f"{obj_item[0]},{obj_item[1]},{obj_item[2]},{obj_item[3]},{obj_item[4]},{obj_item[5]},{obj_item[6]}\n")
    

In [50]:
s_idx = 1
for r_idx in range(5):
    
    print(f"{scene_list[s_idx]} {r_idx}")    
    visualize_area_obj(s_idx, r_idx)
    plt.cla()

s_idx = 3
for r_idx in range(2):
    
    print(f"{scene_list[s_idx]} {r_idx}")    
    visualize_area_obj(s_idx, r_idx)
    plt.cla()
    
s_idx = 4
for r_idx in range(6):
    
    print(f"{scene_list[s_idx]} {r_idx}")    
    visualize_area_obj(s_idx, r_idx)
    plt.cla()
    
s_idx = 5
for r_idx in range(3):
    
    print(f"{scene_list[s_idx]} {r_idx}")    
    visualize_area_obj(s_idx, r_idx)
    plt.cla()
    
s_idx = 6
for r_idx in range(6):
    
    print(f"{scene_list[s_idx]} {r_idx}")    
    visualize_area_obj(s_idx, r_idx)
    plt.cla()
    
plt.close("all")

Beechwood_1_int 0
JSON file consists 97 objects
Area has 22 objects
Beechwood_1_int 1
JSON file consists 97 objects
Area has 19 objects
Beechwood_1_int 2
JSON file consists 97 objects
Area has 10 objects
Beechwood_1_int 3
JSON file consists 97 objects
Area has 14 objects
Beechwood_1_int 4
JSON file consists 97 objects
Area has 21 objects
Ihlen_0_int 0
JSON file consists 49 objects
Area has 22 objects
Ihlen_0_int 1
JSON file consists 49 objects
Area has 6 objects
Ihlen_1_int 0
JSON file consists 109 objects
Area has 18 objects
Ihlen_1_int 1
JSON file consists 109 objects
Area has 12 objects
Ihlen_1_int 2
JSON file consists 109 objects
Area has 28 objects
Ihlen_1_int 3
JSON file consists 109 objects
Area has 15 objects
Ihlen_1_int 4
JSON file consists 109 objects
Area has 16 objects
Ihlen_1_int 5
JSON file consists 109 objects
Area has 16 objects
Merom_0_int 0
JSON file consists 68 objects
Area has 9 objects
Merom_0_int 1
JSON file consists 68 objects
Area has 16 objects
Merom_0_int 2
JS

C:\Users\12871\AppData\Local\Temp\ipykernel_872\2478530071.py:105: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig = plt.figure(figsize=(20, 20))


Merom_1_int 5
JSON file consists 95 objects
Area has 20 objects


#### Visualize windows within area on map (any specific type)

In [37]:
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

import xml.dom.minidom


def read_area_xml(xml_pth):
    """
    Read area information from xml file

    Args:
        xml_pth (str): object xml file path

    Returns:
        list(list(int, int, int, int)): boundary of area
    """
    dom = xml.dom.minidom.parse(xml_pth)
    root = dom.documentElement

    xml_objects = root.getElementsByTagName('object')

    objects_p = []
    for xo in xml_objects:
        
        bbox = xo.getElementsByTagName('bndbox')[0]

        xmin = bbox.getElementsByTagName('xmin')[0].firstChild.data
        ymin = bbox.getElementsByTagName('ymin')[0].firstChild.data
        xmax = bbox.getElementsByTagName('xmax')[0].firstChild.data
        ymax = bbox.getElementsByTagName('ymax')[0].firstChild.data
        
        objects_p.append((int(xmin), int(ymin), int(xmax), int(ymax)))
        
    return objects_p


def read_json_slabel(json_pth, s_label):
    """
    Read object with specific type from json file

    Args:
        json_pth (str): object json file path
        s_label(str): specific semantic type label, eg. window

    Returns:
        list(list(str, float, float)): semantic label and position of objects
    """
    obj_list = []
    
    with open(json_pth, encoding="utf-8") as f:
        
        js_data = json.load(f)
        
        for item in js_data:
            
            # object semantic label
            k1 = "category"
            v1 = item[k1]
            label = v1
            
            if v1 != s_label:
                continue
            
            # object X, Y axis center
            k2 = "center"
            v2 = (item[k2][0], item[k2][1])
            x_center = round(v2[0], 4)
            y_center = round(v2[1], 4)
            
            # object Z axis center and scale
            k3 = "z"
            v3 = (item[k3][0], item[k3][1])
            z_center = round((v3[0] + v3[1]) / 2, 4)
            z_scale = round(abs(v3[0] - v3[1]), 4)
            
            # object X, Y axis scale
            k4 = "raw_pts"
            v4 = item[k4]
            
            x_list = [p[0] for p in v4]
            y_list = [p[1] for p in v4]

            x_max = max(x_list)
            x_min = min(x_list)
            y_max = max(y_list)
            y_min = min(y_list)

            x_scale = round(x_max - x_min, 4)
            y_scale = round(y_max - y_min, 4)
            
            obj_list.append([label, x_center, y_center, x_scale, y_scale, z_center, z_scale])
        
    print(f"JSON file consists {len(obj_list)} {s_label}")
        

    return obj_list  


def visualize_area_slabel(s_idx, r_idx, s_label):
    
    scene_id = scene_list[s_idx]
    
    fig = plt.figure(figsize=(20, 20))
    axes = fig.add_subplot(1, 1, 1)
    
    # Visualize map image
    ds_pth = "E:/Workspace/Datasets/iGibson-dataset/iGibson-map/"
    img_pth = f"{ds_pth}/{scene_id}/{scene_id}.png"
    img = Image.open(img_pth).convert("RGB")
    plt.imshow(img)
    
    # Read area boundary from xml file
    ds_pth = f"E:/Workspace/Datasets/iGibson-dataset/iGibson-area2/"
    xml_pth = f"{ds_pth}/{scene_id}/{r_idx}/{scene_id}.xml"
    xml_data = read_area_xml(xml_pth)[0]
    x_min = xml_data[0]
    y_min = xml_data[1]
    x_max = xml_data[2]
    y_max = xml_data[3]

    # Draw area on scene map
    square_bound = (xml_data[0], xml_data[1], xml_data[2], xml_data[3])
    square = plt.Rectangle(xy = (square_bound[0], square_bound[1]),
                        width = square_bound[2]-square_bound[0],
                        height= square_bound[3]-square_bound[1],
                        edgecolor='blue', 
                        facecolor='none')
    axes.add_patch(square)
    
    # Read object json
    ds_pth = "E:/Workspace/Datasets/iGibson-dataset/iGibson-map/"
    obj_pth = f"{ds_pth}/{scene_id}/{scene_id}.json"
    obj_data = read_json_slabel(obj_pth, s_label)
    
    # Translate object to map coordinate
    obj_sub = []
    
    scene_trans = scene_dict[scene_id]
   
    for obj_item in obj_data:
                
        obj_label = obj_item[0]
        
        x_center = float(obj_item[1]) * scene_trans[2] + scene_trans[0]
        y_center = float(obj_item[2]) * scene_trans[2] + scene_trans[1]
        
        x_scale = float(obj_item[3]) * scene_trans[2]
        y_scale = float(obj_item[4]) * scene_trans[2]
        
        if x_min < x_center < x_max and y_min < y_center < y_max:
            
            rectangle = Rectangle((
                                    x_center - (x_scale / 2), 
                                    y_center - (y_scale / 2)
                                    ), 
                                x_scale, 
                                y_scale, 
                                edgecolor='none', 
                                facecolor='gold')
            
            axes.add_patch(rectangle)
            
            obj_sub.append((obj_item[0], obj_item[1], obj_item[2], obj_item[3], obj_item[4], obj_item[5], obj_item[6]))

    print(f"Area has {len(obj_sub)} {s_label}")

    ds_pth = "E:/Workspace/Datasets/iGibson-dataset/iGibson-area/"
    plt.savefig(f"{ds_pth}/{scene_id}/{r_idx}/area_{s_label}.png")
    
    # Output result to txt file
    ds_pth = "E:/Workspace/Datasets/iGibson-dataset/iGibson-area/"
    slabel_map_pth = f"{ds_pth}/{scene_id}/{r_idx}/map_{s_label}.txt"
    slabel_map_fp = open(slabel_map_pth, "w", encoding="utf-8")

    for obj_item in obj_sub:
        slabel_map_fp.write(f"{obj_item[0]},{obj_item[1]},{obj_item[2]},{obj_item[3]},{obj_item[4]},{obj_item[5]},{obj_item[6]}\n")
        
    

In [38]:
s_label = "window"

s_idx = 1
for r_idx in range(5):
    
    print(f"{scene_list[s_idx]} {r_idx}")    
    visualize_area_slabel(s_idx, r_idx, s_label)
    plt.cla()

s_idx = 3
for r_idx in range(2):
    
    print(f"{scene_list[s_idx]} {r_idx}")    
    visualize_area_slabel(s_idx, r_idx, s_label)
    plt.cla()
    
s_idx = 4
for r_idx in range(6):
    
    print(f"{scene_list[s_idx]} {r_idx}")    
    visualize_area_slabel(s_idx, r_idx, s_label)
    plt.cla()
    
s_idx = 5
for r_idx in range(3):
    
    print(f"{scene_list[s_idx]} {r_idx}")    
    visualize_area_slabel(s_idx, r_idx, s_label)
    plt.cla()
    
s_idx = 6
for r_idx in range(6):
    
    print(f"{scene_list[s_idx]} {r_idx}")    
    visualize_area_slabel(s_idx, r_idx, s_label)
    plt.cla()
    
plt.close("all")

Beechwood_1_int 0
JSON file consists 8 window
Area has 1 window
Beechwood_1_int 1
JSON file consists 8 window
Area has 1 window
Beechwood_1_int 2
JSON file consists 8 window
Area has 1 window
Beechwood_1_int 3
JSON file consists 8 window
Area has 1 window
Beechwood_1_int 4
JSON file consists 8 window
Area has 2 window
Ihlen_0_int 0
JSON file consists 3 window
Area has 1 window
Ihlen_0_int 1
JSON file consists 3 window
Area has 1 window
Ihlen_1_int 0
JSON file consists 12 window
Area has 3 window
Ihlen_1_int 1
JSON file consists 12 window
Area has 2 window
Ihlen_1_int 2
JSON file consists 12 window
Area has 1 window
Ihlen_1_int 3
JSON file consists 12 window
Area has 1 window
Ihlen_1_int 4
JSON file consists 12 window
Area has 2 window
Ihlen_1_int 5
JSON file consists 12 window
Area has 2 window
Merom_0_int 0
JSON file consists 6 window
Area has 2 window
Merom_0_int 1
JSON file consists 6 window
Area has 2 window
Merom_0_int 2
JSON file consists 6 window
Area has 1 window
Merom_1_int 0


C:\Users\12871\AppData\Local\Temp\ipykernel_872\3623926590.py:104: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig = plt.figure(figsize=(20, 20))


Merom_1_int 5
JSON file consists 10 window
Area has 3 window


#### Visualize wall mask within area on map

In [29]:
from PIL import Image
import matplotlib.pyplot as plt
import xml.dom.minidom


def read_wall_xml(xml_pth):

    dom = xml.dom.minidom.parse(xml_pth)
    root = dom.documentElement

    xml_objects = root.getElementsByTagName('object')
    # print(f"{len(xml_objects)} objects")

    objects_p = []
    for xo in xml_objects:
        
        bbox = xo.getElementsByTagName('bndbox')[0]

        xmin = bbox.getElementsByTagName('xmin')[0].firstChild.data
        ymin = bbox.getElementsByTagName('ymin')[0].firstChild.data
        xmax = bbox.getElementsByTagName('xmax')[0].firstChild.data
        ymax = bbox.getElementsByTagName('ymax')[0].firstChild.data
        
        objects_p.append((int(xmin), int(ymin), int(xmax), int(ymax)))
        
    return objects_p


def visualize_area_mask(s_idx, r_idx):

    scene_id = scene_list[s_idx]

    fig = plt.figure(figsize=(20, 20))
    axes = fig.add_subplot(1, 1, 1)

    # Visualize map image
    ds_pth = "E:/Workspace/Datasets/iGibson-dataset/iGibson-map/"
    img_pth = f"{ds_pth}/{scene_id}/{scene_id}.png"
    img = Image.open(img_pth).convert("RGB")
    plt.imshow(img)

    # Read wall mask from xml file
    ds_pth = "E:/Workspace/Datasets/iGibson-dataset/iGibson-area/"
    xml_pth = f"{ds_pth}/{scene_id}/{r_idx}/mask.xml"
    xml_data = read_wall_xml(xml_pth)
    
    # Visualize wall mask
    for wall_mask in xml_data:
        x_min = wall_mask[0]
        y_min = wall_mask[1]
        x_max = wall_mask[2]
        y_max = wall_mask[3]

        square_bound = (x_min, y_min, x_max, y_max)
        square = plt.Rectangle(xy = (square_bound[0], square_bound[1]),
                            width = square_bound[2]-square_bound[0],
                            height= square_bound[3]-square_bound[1],
                            edgecolor="blue",
                            facecolor="blue")
        axes.add_patch(square)

    # Output result to txt file
    ds_pth = "E:/Workspace/Datasets/iGibson-dataset/iGibson-area/"
    wmap_pth = f"{ds_pth}/{scene_id}/{r_idx}/wmap.txt"
    wmap_fp = open(wmap_pth, "w", encoding="utf-8")
    
    scene_trans = scene_dict[scene_id]
    
    for wall_mask in xml_data:
        
        x_min = wall_mask[0]
        y_min = wall_mask[1]
        x_max = wall_mask[2]
        y_max = wall_mask[3]
        
        x = (x_min + x_max) / 2
        x = (x - scene_trans[0]) / scene_trans[2]
        x = round(x, 4)
        y = (y_min + y_max) / 2
        y = (y - scene_trans[1]) / scene_trans[2]
        y = round(y, 4)
        
        length = round(abs(x_max - x_min) / 100, 4)
        width = round(abs(y_max - y_min) / 100, 4)
        
        wmap_fp.write(f"{x},{y},{length},{width}\n")
        
    wmap_fp.close()

    ds_pth = "E:/Workspace/Datasets/iGibson-dataset/iGibson-area/"
    plt.savefig(f"{ds_pth}/{scene_id}/{r_idx}/area_mask.png")


In [30]:
s_idx = 1
for r_idx in range(5):
    
    print(f"{scene_list[s_idx]} {r_idx}")    
    visualize_area_mask(s_idx, r_idx)
    plt.cla()

s_idx = 3
for r_idx in range(2):
    
    print(f"{scene_list[s_idx]} {r_idx}")    
    visualize_area_mask(s_idx, r_idx)
    plt.cla()
    
s_idx = 4
for r_idx in range(6):
    
    print(f"{scene_list[s_idx]} {r_idx}")    
    visualize_area_mask(s_idx, r_idx)
    plt.cla()
    
s_idx = 5
for r_idx in range(3):
    
    print(f"{scene_list[s_idx]} {r_idx}")    
    visualize_area_mask(s_idx, r_idx)
    plt.cla()
    
s_idx = 6
for r_idx in range(6):
    
    print(f"{scene_list[s_idx]} {r_idx}")    
    visualize_area_mask(s_idx, r_idx)
    plt.cla()
    
plt.close("all")

Beechwood_1_int 0
Beechwood_1_int 1
Beechwood_1_int 2
Beechwood_1_int 3
Beechwood_1_int 4
Ihlen_0_int 0
Ihlen_0_int 1
Ihlen_1_int 0
Ihlen_1_int 1
Ihlen_1_int 2
Ihlen_1_int 3
Ihlen_1_int 4
Ihlen_1_int 5
Merom_0_int 0
Merom_0_int 1
Merom_0_int 2
Merom_1_int 0
Merom_1_int 1
Merom_1_int 2
Merom_1_int 3
Merom_1_int 4


C:\Users\12871\AppData\Local\Temp\ipykernel_872\4041637734.py:33: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig = plt.figure(figsize=(20, 20))


Merom_1_int 5


In [43]:
from PIL import Image
import matplotlib.pyplot as plt
import xml.dom.minidom


def read_xml(xml_pth):

    dom = xml.dom.minidom.parse(xml_pth)
    root = dom.documentElement

    xml_objects = root.getElementsByTagName('object')
    # print(f"{len(xml_objects)} objects")

    objects_p = []
    for xo in xml_objects:
        
        bbox = xo.getElementsByTagName('bndbox')[0]

        xmin = bbox.getElementsByTagName('xmin')[0].firstChild.data
        ymin = bbox.getElementsByTagName('ymin')[0].firstChild.data
        xmax = bbox.getElementsByTagName('xmax')[0].firstChild.data
        ymax = bbox.getElementsByTagName('ymax')[0].firstChild.data
        
        objects_p.append((int(xmin), int(ymin), int(xmax), int(ymax)))
        
    return objects_p


def visualize_area_mask(s_idx, r_idx):

    scene_id = scene_list[s_idx]

    fig = plt.figure(figsize=(20, 20))
    axes = fig.add_subplot(1, 1, 1)

    # Visualize map image
    ds_pth = "E:/Workspace/Datasets/iGibson-dataset/iGibson-map/"
    img_pth = f"{ds_pth}/{scene_id}/{scene_id}.png"
    img = Image.open(img_pth).convert("RGB")
    plt.imshow(img)

    
    # Read wall mask from xml file
    ds_pth = "E:/Workspace/Datasets/iGibson-dataset/iGibson-area/"
    xml_pth = f"{ds_pth}/{scene_id}/{r_idx}/mask.xml"
    xml_data = read_xml(xml_pth)
    
    # Visualize wall mask
    for wall_mask in xml_data:
        x_min = wall_mask[0]
        y_min = wall_mask[1]
        x_max = wall_mask[2]
        y_max = wall_mask[3]

        square_bound = (x_min, y_min, x_max, y_max)
        square = plt.Rectangle(xy = (square_bound[0], square_bound[1]),
                            width = square_bound[2]-square_bound[0],
                            height= square_bound[3]-square_bound[1],
                            edgecolor="blue",
                            facecolor="blue")
        axes.add_patch(square)

    # Transform wall mask to bbox
    ds_pth = "E:/Workspace/Datasets/iGibson-dataset/iGibson-area/"
    wmap_pth = f"{ds_pth}/{scene_id}/{r_idx}/wmap.txt"
    wmap_fp = open(wmap_pth, "w", encoding="utf-8")
    
    scene_trans = scene_dict[scene_id]
    
    for wall_mask in xml_data:
        x_min = wall_mask[0]
        y_min = wall_mask[1]
        x_max = wall_mask[2]
        y_max = wall_mask[3]
        
        x = (x_min + x_max) / 2
        x = (x - scene_trans[0]) / scene_trans[2]
        x = round(x, 4)
        y = (y_min + y_max) / 2
        y = (y - scene_trans[1]) / scene_trans[2]
        y = round(y, 4)
        
        length = round(abs(x_max - x_min) / 100, 4)
        width = round(abs(y_max - y_min) / 100, 4)
        
        wmap_fp.write(f"{x},{y},{length},{width}\n")
        
    wmap_fp.close()

    # Read window mask from txt file
    ds_pth = "E:\Workspace\Datasets\iGibson-dataset\iGibson-area2/"
    window_pth = f"{ds_pth}/{scene_id}/{r_idx}/ad_window.txt"
    window_fp = open(window_pth, encoding="utf-8")
    window_data = window_fp.readlines()
    
    scene_trans = scene_dict[scene_id]
    
    for window in window_data:
        
        window = window.replace("\n", "").split(",")
        window_x_center = float(window[0]) * scene_trans[2] + scene_trans[0]
        window_y_center = float(window[1]) * scene_trans[2] + scene_trans[1]
        window_x_scale = float(window[2]) * scene_trans[2]
        window_y_scale = float(window[3]) * scene_trans[2]

        rect = Rectangle((
            window_x_center - window_x_scale / 2,
            window_y_center - window_y_scale / 2),
            window_x_scale,
            window_y_scale,
            edgecolor='red',
            facecolor="red"
                         )
        axes.add_patch(rect)

    # Read sample positions from txt
    ds_pth = "E:/Workspace/Datasets/iGibson-dataset/iGibson-area/"
    gt_pth = f"{ds_pth}/{scene_id}/{r_idx}/traj_sub.txt"
    gt_positions = []

    gt_fp = open(gt_pth, encoding="utf-8")
    for gt_p in gt_fp.readlines():
        gt_p = ((gt_p).replace("\n", "")).split(",")
        gt_positions.append((gt_p[0], gt_p[1]))

    # Translate sample position to map coordinate
    scene_trans = scene_dict[scene_id]

    traj_x = []
    traj_y = []
    
    for gt_p in gt_positions:
        
        pos_x = float(gt_p[0]) * scene_trans[2] + scene_trans[0]
        pos_y = float(gt_p[1]) * scene_trans[2] + scene_trans[1]
        traj_x.append(pos_x)
        traj_y.append(pos_y)    
        
    plt.scatter(traj_x, traj_y)
    
    ds_pth = "E:/Workspace/Datasets/iGibson-dataset/iGibson-area2/"
    
    plt.savefig(f"{ds_pth}/{scene_id}/{r_idx}/area_mask.png")


In [44]:
s_idx = 1
for r_idx in range(5):
    visualize_area_mask(s_idx, r_idx)
plt.cla()

s_idx = 5
for r_idx in range(2):
    visualize_area_mask(s_idx, r_idx)
plt.cla()
    
s_idx = 6
for r_idx in range(6):
    visualize_area_mask(s_idx, r_idx)
plt.cla()
    
s_idx = 7
for r_idx in range(3):
    visualize_area_mask(s_idx, r_idx)
plt.cla()
    
s_idx = 8
for r_idx in range(6):
    visualize_area_mask(s_idx, r_idx)
plt.cla()
    
plt.close("all")

C:\Users\12871\AppData\Local\Temp\ipykernel_964\1099147174.py:33: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig = plt.figure(figsize=(20, 20))


#### Select windows within area

In [41]:
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

import xml.dom.minidom


def read_area_xml(xml_pth):

    dom = xml.dom.minidom.parse(xml_pth)
    root = dom.documentElement

    xml_objects = root.getElementsByTagName('object')
    # print(f"{len(xml_objects)} objects")

    objects_p = []
    for xo in xml_objects:
        
        bbox = xo.getElementsByTagName('bndbox')[0]

        xmin = bbox.getElementsByTagName('xmin')[0].firstChild.data
        ymin = bbox.getElementsByTagName('ymin')[0].firstChild.data
        xmax = bbox.getElementsByTagName('xmax')[0].firstChild.data
        ymax = bbox.getElementsByTagName('ymax')[0].firstChild.data
        
        objects_p.append((int(xmin), int(ymin), int(xmax), int(ymax)))
        
    return objects_p


def area_window(s_idx, r_idx):
    
    fig = plt.figure(figsize=(20, 20))
    axes = fig.add_subplot(1, 1, 1)
    
    scene_id = scene_list[s_idx]
    scene_trans = scene_dict[scene_id]
    
    # Read area from xml file
    area_pth = f"E:\Workspace\Datasets\iGibson-dataset\iGibson-area2\{scene_id}/{r_idx}/{scene_id}.xml"
    
    xml_data = read_area_xml(area_pth)[0]
    x_min = (xml_data[0] - scene_trans[0]) / scene_trans[2]
    y_min = (xml_data[1] - scene_trans[1]) / scene_trans[2]
    x_max = (xml_data[2] - scene_trans[0]) / scene_trans[2]
    y_max = (xml_data[3] - scene_trans[1]) / scene_trans[2]
    
    square_bound = (x_min, y_min, x_max, y_max)
    square = plt.Rectangle(xy = (square_bound[0], square_bound[1]),
                        width = square_bound[2]-square_bound[0],
                        height= square_bound[3]-square_bound[1],
                        edgecolor="red",
                        facecolor="none")
    axes.add_patch(square)
    
    # Read windows data from txt file
    window_pth = f"E:\Workspace\Datasets\iGibson-dataset\iGibson-area2\{scene_id}/{r_idx}/window.txt"
    window_fp = open(window_pth, encoding="utf-8")
    window_data = window_fp.readlines()
    
    area_window_pth = f"E:\Workspace\Datasets\iGibson-dataset\iGibson-area2\{scene_id}/{r_idx}/ad_window.txt"
    area_window_fp = open(area_window_pth, "w", encoding="utf-8")
    
    window_in_area = []
    for window in window_data:
        
        window_cp = window.replace("\n", "").split(",")
        x_center = float(window_cp[0])
        y_center = float(window_cp[1])
        x_scale = float(window_cp[2])
        y_scale = float(window_cp[3])
        
        
        if x_min < x_center < x_max and y_min < y_center < y_max:
            window_in_area.append([x_center, y_center, x_scale, y_scale])
            
            area_window_fp.write(window)
    
    # Visualize windows data
    map_image = Image.open(f"E:\Workspace\Datasets\iGibson-dataset\iGibson-area2\{scene_id}/{r_idx}/{scene_id}.png").convert("RGB")
    plt.imshow(map_image)
    
    for window in window_in_area:
        
        window_x_center = window[0] * scene_trans[2] + scene_trans[0]
        window_y_center = window[1] * scene_trans[2] + scene_trans[1]
        window_x_scale = window[2] * scene_trans[2]
        window_y_scale = window[3] * scene_trans[2]
        
        rect = Rectangle((
            window_x_center - window_x_scale / 2,
            window_y_center - window_y_scale / 2),
            window_x_scale,
            window_y_scale,
            edgecolor='none',
            facecolor="red"
                         )
        axes.add_patch(rect)
        
    plt.savefig(f"E:\Workspace\Datasets\iGibson-dataset\iGibson-area2\{scene_id}/{r_idx}/ad_window.png")
    # plt.show()
    

In [42]:
s_idx = 1
for r_idx in range(5):
    area_window(s_idx, r_idx)
    plt.cla()

s_idx = 5
for r_idx in range(2):
    area_window(s_idx, r_idx)
    plt.cla()
    
s_idx = 6
for r_idx in range(6):
    area_window(s_idx, r_idx)
    plt.cla()
    
s_idx = 7
for r_idx in range(3):
    area_window(s_idx, r_idx)
    plt.cla()
    
s_idx = 8
for r_idx in range(6):
    area_window(s_idx, r_idx)
    plt.cla()
    
plt.close("all")

C:\Users\12871\AppData\Local\Temp\ipykernel_964\2665986239.py:33: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig = plt.figure(figsize=(20, 20))
